In [ ]:
!pip install python-dotenv
!pip install openai
!pip install langchain


* ConversationBufferMemory


In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [ ]:
style = """```child aged between 9 to 13 who finds not much attention from his parents.\
childish and unawared tone with not accurate american english\
puctuation should also be not correct.```
"""

prompt_template =  """Behave as a child whose age is between 9 to 13. Your task is to talk to\
child sexual predetor and answer them like child and try get information like their address, name,or any useful information\
but dont rush and dont ask everything in one response . wait for more messages and use short size respons\
but try to keep him in the conversation\
your talking style is ``` """+style+""" ``` \nCurrent conversation:\n{history}\nLast line:\nHuman: {input}\nYou:"""

prompt_template_02 =  """Behave as a child whose age is between 9 to 13. Your task is to talk to\
child sexual predetor and answer them like child and try get information like their address, name,or any useful information\
but dont rush and dont ask everything in one response . wait for more messages and use short size respons\
but try to keep him in the conversation\
your talking style is """+style+"""\nCurrent conversation:\n{history}\nLast line:\nPredator: {input}\n"""

# Now we can override it and set it to "AI Assistant"
from langchain.prompts.prompt import PromptTemplate

# template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

# Current conversation:
# {history}
# Human: {input}
# AI Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=prompt_template_02)
llm = ChatOpenAI(temperature=0.0, model=llm_model)

memory = ConversationBufferMemory(ai_prefix="Kid",human_prefix='predator')
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=memory,
)

In [ ]:
conversation.predict(input="Hi, my name is Jack")

In [ ]:
conversation.predict(input="Hey, I noticed we have a lot in common, and I really enjoy chatting with you. You're so mature for your age")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="You're incredibly smart and talented. I've never met anyone like you before")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
conversation.predict(input="I can buy you that game you've been wanting. Just tell me where you live, and I'll send it to you as a surprise")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

### Parse the LLM output string into a Python dictionary

In [ ]:
name_schema = ResponseSchema(name="name",
                             description="it is the name of predator\
                             Answer True if yes,\
                             None if not or unknown.")
address_schema = ResponseSchema(name="address",
                                      description="addresses metioned by predator\
                                      If this \
                                      information is not found,\
                                      output None.")
readiness_schema = ResponseSchema(name="readiness",
                                    description="if predator is ready to meet\
                                    0 or 1.")

response_schemas = [name_schema,
                    address_schema,
                    readiness_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
from langchain.prompts import ChatPromptTemplate

# prompt_template = ChatPromptTemplate.from_template(template_string)

review_template_2 = """\
For the following conversation, extract the following information:

name : "it is the name of predator Answer True if yes,None if not or unknown."

address: "addresses metioned by predator If this information is not found, output None."

readiness: "if predator is ready to meet , output 0 or 1."

conversation: {conversation}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

full_conversation = prompt.format_messages(conversation=memory.buffer,
                                format_instructions=format_instructions)

In [ ]:
print(full_conversation[0].content)

In [ ]:
response = llm(full_conversation)

In [ ]:
print(response.content)